# 5 pandas入门

pandas 是本书后续内容的首选库。它含有使数据清洗和分析工作变得更快更简 单的数据结构和操作工具。
pandas 经常和其它工具一同使用，如数值计算工具 NumPy 和 SciPy，分析库 statsmodels 和 scikit-learn，和数据可视化库 matplotlib。pandas 是基于 NumPy 数组构建的，特别是基于数组的函数和不使 用 for 循环的数据处理。
虽然 pandas 采用了大量的 NumPy 编码风格，但二者最大的不同是 pandas 是专 门为处理表格和混杂数据设计的。而 NumPy 更适合处理统一的数值数组数据。

In [2]:
import pandas as pd
from pandas import Series, DataFrame

## 5.1 pandas的数据结构介绍

要使用 pandas，你首先就得熟悉它的两个主要数据结构：Series 和 DataFrame。虽然它们并不能解决所有问题，但它们为大多数应用提供了一种可 靠的、易于使用的基础

### Series

Series 是一种类似于一维数组的对象，它由一组数据（各种 NumPy 数据类型） 以及一组与之相关的数据标签（即索引）组成。仅由一组数据即可产生最简单 的 Series：

In [3]:
obj = pd.Series([4, 7, -5, 3])
obj

0    4
1    7
2   -5
3    3
dtype: int64

Series 的字符串表现形式为：索引在左边，值在右边。由于我们没有为数据指 定索引，于是会自动创建一个 0 到 N-1（N 为数据的长度）的整数型索引。你可 以通过 Series 的 values 和 index 属性获取其数组表示形式和索引对象

In [4]:
obj.values

array([ 4,  7, -5,  3])

In [5]:
obj.index

RangeIndex(start=0, stop=4, step=1)

通常，我们希望所创建的 Series 带有一个可以对各个数据点进行标记的索引

In [6]:
obj2 = pd.Series([4, 7, -5, 3], index=['d', 'b', 'a', 'c'])
obj2

d    4
b    7
a   -5
c    3
dtype: int64

与普通 NumPy 数组相比，你可以通过索引的方式选取 Series 中的单个或一组 值：

In [7]:
obj2['a']

-5

In [8]:
obj2['d'] = 6
obj2

d    6
b    7
a   -5
c    3
dtype: int64

In [9]:
obj2[['c', 'a', 'd']]

c    3
a   -5
d    6
dtype: int64

使用 NumPy 函数或类似 NumPy 的运算（如根据布尔型数组进行过滤、标量乘 法、应用数学函数等）都会保留索引值的链接：

In [10]:
obj2[obj2  > 0]

d    6
b    7
c    3
dtype: int64

In [11]:
obj2 * 2

d    12
b    14
a   -10
c     6
dtype: int64

In [12]:
import numpy as np
np.exp(obj2)

d     403.428793
b    1096.633158
a       0.006738
c      20.085537
dtype: float64

In [13]:
# 还可以将 Series 看成是一个定长的有序字典，因为它是索引值到数据值的一个 映射。它可以用在许多原本需要字典参数的函数中：
'b' in obj2

True

In [14]:
# 如果数据被存放在一个 Python 字典中，也可以直接通过这个字典来创建 Series：
sdata = {'Ohio': 35000, 'Texas': 71000, 'Oregon': 16000, 'Utah': 5000}
obj3 = pd.Series(sdata)
obj3

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64

In [15]:
obj3.index

Index(['Ohio', 'Texas', 'Oregon', 'Utah'], dtype='object')

In [16]:
# 如果只传入一个字典，则结果 Series 中的索引就是原字典的键（有序排列）。 你可以传入排好序的字典的键以改变顺序：
states = ['California', 'Ohio', 'Oregon', 'Texas']
obj4 = pd.Series(sdata,index=states)
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

In [17]:
# 在这个例子中，sdata 中跟 states 索引相匹配的那 3 个值会被找出来并放到相 应的位置上，但由于"California"所对应的 sdata 值找不到，所以其结果就为 NaN（即“非数字”（not a number），在 pandas 中，它用于表示缺失或 NA 值）。因为‘Utah’不在 states 中，它被从结果中除去

使用缺失（missing）或 NA 表示缺失数据。pandas 的 isnull 和 notnull 函数可用于检测缺失数据

In [18]:
pd.isnull(obj4)

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

In [19]:
pd.notnull(obj4)

California    False
Ohio           True
Oregon         True
Texas          True
dtype: bool

In [20]:
# Series 也有类似的实例方法
obj4.isnull()

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

对于许多应用而言，Series 最重要的一个功能是，它会根据运算的索引标签自 动对齐数据

In [21]:
obj3

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64

In [22]:
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

In [23]:
obj3 + obj4

California         NaN
Ohio           70000.0
Oregon         32000.0
Texas         142000.0
Utah               NaN
dtype: float64

Series 对象本身及其索引都有一个 name 属性，该属性跟 pandas 其他的关键功 能关系非常密切：

In [24]:
obj4.name = 'population'

In [25]:
obj4.index.name = 'state'

In [26]:
obj4

state
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
Name: population, dtype: float64

Series 的索引可以通过赋值的方式就地修改

In [27]:
obj

0    4
1    7
2   -5
3    3
dtype: int64

In [28]:
obj.index = ['Bob', 'Steve', 'Jeff', 'Ryan']

In [29]:
obj

Bob      4
Steve    7
Jeff    -5
Ryan     3
dtype: int64

### DataFrame

DataFrame 是一个表格型的数据结构，它含有一组有序的列，每列可以是不同 的值类型（数值、字符串、布尔值等）。DataFrame 既有行索引也有列索引， 它可以被看做由 Series 组成的字典（共用同一个索引）。DataFrame 中的数据 是以一个或多个二维块存放的（而不是列表、字典或别的一维数据结构）。有 关 DataFrame 内部的技术细节远远超出了本书所讨论的范围。

笔记：虽然 DataFrame 是以二维结构保存数据的，但你仍然可以轻松地将其表 示为更高维度的数据（层次化索引的表格型结构，这是 pandas 中许多高级数据 处理功能的关键要素，我们会在第 8 章讨论这个问题）。

建 DataFrame 的办法有很多，最常用的一种是直接传入一个由等长列表或 NumPy 数组组成的字典：

In [30]:
data = {'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada', 'Nevada'],
'year': [2000, 2001, 2002, 2001, 2002, 2003],
'pop': [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]}
frame = pd.DataFrame(data)
frame
#结果 DataFrame 会自动加上索引（跟 Series 一样），且全部列会被有序排列

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9
5,Nevada,2003,3.2


In [31]:
# 对于特别大的 DataFrame，head 方法会选取前五行
frame.head()

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9


In [32]:
# 如果指定了列序列，则 DataFrame 的列就会按照指定顺序进行排列：
pd.DataFrame(data, columns=['year', 'state', 'pop'])

,year,state,pop
0,2000,Ohio,1.5
1,2001,Ohio,1.7
2,2002,Ohio,3.6
3,2001,Nevada,2.4
4,2002,Nevada,2.9
5,2003,Nevada,3.2


In [33]:
# 如果传入的列在数据中找不到，就会在结果中产生缺失值：
frame2 = pd.DataFrame(data, columns=['year', 'state', 'pop', 'debt'],
index=['one', 'two', 'three', 'four', 'five', 'six'])
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,NaN
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,NaN
five,2002,Nevada,2.9,NaN
six,2003,Nevada,3.2,NaN


In [34]:
frame2.columns 
# 通过类似字典标记的方式或属性的方式，可以将 DataFrame 的列获取为一个 Series：

Index(['year', 'state', 'pop', 'debt'], dtype='object')

In [35]:
frame2.year

one      2000
two      2001
three    2002
four     2001
five     2002
six      2003
Name: year, dtype: int64

笔记：IPython 提供了类似属性的访问（即 frame2.year）和 tab 补全。 frame2[column]适用于任何列的名，但是 frame2.column 只有在列名是一个合 理的 Python 变量名时才适用。

注意，返回的 Series 拥有原 DataFrame 相同的索引，且其 name 属性也已经被 相应地设置好了。

行也可以通过位置或名称的方式进行获取，比如用 loc 属性（稍后将对此进行 详细讲解）：

In [36]:
frame2.loc['three']

year     2002
state    Ohio
pop       3.6
debt      NaN
Name: three, dtype: object

列可以通过赋值的方式进行修改。例如，我们可以给那个空的"debt"列赋上一 个标量值或一组值：

In [37]:
frame2['debt'] = 16.5
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,16.5
two,2001,Ohio,1.7,16.5
three,2002,Ohio,3.6,16.5
four,2001,Nevada,2.4,16.5
five,2002,Nevada,2.9,16.5
six,2003,Nevada,3.2,16.5


In [38]:
frame2['debt'] = np.arange(6.)
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,0.0
two,2001,Ohio,1.7,1.0
three,2002,Ohio,3.6,2.0
four,2001,Nevada,2.4,3.0
five,2002,Nevada,2.9,4.0
six,2003,Nevada,3.2,5.0


将列表或数组赋值给某个列时，其长度必须跟 DataFrame 的长度相匹配。如果 赋值的是一个 Series，就会精确匹配 DataFrame 的索引，所有的空位都将被填 上缺失值：

In [39]:
val = pd.Series([-1.2, -1.5, -1.7], index=['two', 'four', 'five'])
val

two    -1.2
four   -1.5
five   -1.7
dtype: float64

In [40]:
frame2['debt'] = val
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,-1.2
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,-1.5
five,2002,Nevada,2.9,-1.7
six,2003,Nevada,3.2,NaN


为不存在的列赋值会创建出一个新列。关键字 del 用于删除列。

In [41]:
# 作为 del 的例子，我先添加一个新的布尔值的列，state 是否为'Ohio'：
frame2['eastern'] = frame2.state == 'Ohio'
frame2

,year,state,pop,debt,eastern
one,2000,Ohio,1.5,NaN,True
two,2001,Ohio,1.7,-1.2,True
three,2002,Ohio,3.6,NaN,True
four,2001,Nevada,2.4,-1.5,False
five,2002,Nevada,2.9,-1.7,False
six,2003,Nevada,3.2,NaN,False


注意：不能用 frame2.eastern 创建新的列。

In [42]:
# del 方法可以用来删除这列：
del frame2['eastern']

In [43]:
frame2.columns

Index(['year', 'state', 'pop', 'debt'], dtype='object')

注意：通过索引方式返回的列只是相应数据的视图而已，并不是副本。因此， 对返回的 Series 所做的任何就地修改全都会反映到源 DataFrame 上。通过 Series 的 copy 方法即可指定复制列。

In [44]:
#另一种常见的数据形式是嵌套字典
pop = {'Nevada': {2001: 2.4, 2002: 2.9}, 'Ohio': {2000: 1.5, 2001: 1.7, 2002: 3.6}}

In [45]:
# 如果嵌套字典传给 DataFrame，pandas 就会被解释为：外层字典的键作为列， 内层键则作为行索引：
frame3 = pd.DataFrame(pop)
frame3

,Nevada,Ohio
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


你也可以使用类似 NumPy 数组的方法，对 DataFrame 进行转置（交换行和 列）：

In [46]:
frame3.T

,2000,2001,2002
Nevada,NaN,2.4,2.9
Ohio,1.5,1.7,3.6


In [47]:
# 内层字典的键会被合并、排序以形成最终的索引。如果明确指定了索引，则不 会这样：
pd.DataFrame(pop, index=[2001, 2002, 2003])

,Nevada,Ohio
2001,2.4,1.7
2002,2.9,3.6
2003,NaN,NaN


In [48]:
# 由 Series 组成的字典差不多也是一样的用法：
pdata = {'Ohio': frame3['Ohio'][:-1], 'Nevada': frame3['Nevada'][:2]}
pd.DataFrame(pdata)

,Ohio,Nevada
2000,1.5,NaN
2001,1.7,2.4


In [49]:
# 如果设置了 DataFrame 的 index 和 columns 的 name 属性，则这些信息也会被显 示出来：
frame3.index.name = 'year'
frame3.columns.name = 'state'
frame3

state,Nevada,Ohio
year,,
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


跟 Series 一样，values 属性也会以二维 ndarray 的形式返回 DataFrame 中的 数据：

In [50]:
frame3.values

array([[nan, 1.5],
       [2.4, 1.7],
       [2.9, 3.6]])

In [51]:
frame3.values.shape

(3, 2)

如果 DataFrame 各列的数据类型不同，则值数组的 dtype 就会选用能兼容所有 列的数据类型：

In [52]:
frame2.values

array([[2000, 'Ohio', 1.5, nan],
       [2001, 'Ohio', 1.7, -1.2],
       [2002, 'Ohio', 3.6, nan],
       [2001, 'Nevada', 2.4, -1.5],
       [2002, 'Nevada', 2.9, -1.7],
       [2003, 'Nevada', 3.2, nan]], dtype=object)

In [53]:
frame2.values.shape

(6, 4)

### 索引对象

pandas 的索引对象负责管理轴标签和其他元数据（比如轴名称等）。构建 Series 或 DataFrame 时，所用到的任何数组或其他序列的标签都会被转换成一 个 Index：

In [54]:
obj = pd.Series(range(3),  index=['a', 'b', 'c'])
obj

a    0
b    1
c    2
dtype: int64

In [55]:
index = obj.index
index

Index(['a', 'b', 'c'], dtype='object')

In [56]:
index[1:]

Index(['b', 'c'], dtype='object')

Index 对象是不可变的，因此用户不能对其进行修改：

In [57]:
index[1] = 'd' # TypeError: Index does not support mutable operations

TypeError: Index does not support mutable operations

不可变可以使 Index 对象在多个数据结构之间安全共享

In [ ]:
labels = pd.Index(np.arange(3))
labels

In [ ]:
obj2 = pd.Series([1.5,-2.6,0], index=labels)
obj2

In [ ]:
obj2.index is labels

注意：虽然用户不需要经常使用 Index 的功能，但是因为一些操作会生成包含 被索引化的数据，理解它们的工作原理是很重要的。

In [ ]:
# 除了类似于数组，Index 的功能也类似一个固定大小的集合：
frame3

In [ ]:
frame3.columns

In [ ]:
'Ohio' in frame3.columns

In [ ]:
2003 in frame3.columns

In [ ]:
# 与 python 的集合不同，pandas 的 Index 可以包含重复的标签
dup_labels = pd.Index(['foo', 'foo', 'bar', 'bar'])
dup_labels

选择重复的标签，会显示所有的结果。

每个索引都有一些方法和属性，它们可用于设置逻辑并回答有关该索引所包含 的数据的常见问题

## 5.2 基本功能

本节中，我将介绍操作 Series 和 DataFrame 中的数据的基本手段。后续章节将 更加深入地挖掘 pandas 在数据分析和处理方面的功能。本书不是 pandas 库的 详尽文档，主要关注的是最重要的功能，那些不大常用的内容（也就是那些更 深奥的内容）就交给你自己去摸索吧

### 重新索引

pandas 对象的一个重要方法是 reindex，其作用是创建一个新对象，它的数据 符合新的索引。看下面的例子

In [ ]:
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])
obj

In [ ]:
#用该 Series 的 reindex 将会根据新索引进行重排。如果某个索引值当前不存 在，就引入缺失值：
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])
obj2

In [ ]:
# 对于时间序列这样的有序数据，重新索引时可能需要做一些插值处理。method 选项即可达到此目的，例如，使用 ffill 可以实现前向值填充
obj3 = pd.Series(['blue', 'purple', 'yellow'], index=[0, 2, 4])
obj3

In [ ]:
obj3.reindex(range(6), method='ffill')

借助 DataFrame，reindex 可以修改（行）索引和列。只传递一个序列时，会重 新索引结果的行：

In [ ]:
frame = pd.DataFrame(np.arange(9).reshape((3, 3)), 
                     index=['a', 'c', 'd'], 
                     columns=['Ohio', 'Texas', 'California'])
frame

In [ ]:
frame2 = frame.reindex(['a', 'b', 'c', 'd'])
frame2

In [ ]:
# 列可以用 columns 关键字重新索引：
states = ['Texas', 'Utah', 'California']

In [ ]:
frame.reindex(columns=states)

### 丢弃指定轴上的项

丢弃某条轴上的一个或多个项很简单，只要有一个索引数组或列表即可。由于 需要执行一些数据整理和集合逻辑，所以 drop 方法返回的是一个在指定轴上删 除了指定值的新对象

In [ ]:
obj = pd.Series(np.arange(5.), index=['a', 'b', 'c', 'd', 'e'])
obj

In [ ]:
new_obj = obj.drop('c')
new_obj

In [ ]:
obj.drop(['d', 'c'])

对于 DataFrame，可以删除任意轴上的索引值。为了演示，先新建一个 DataFrame 例子：

In [ ]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)), 
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

In [ ]:
# 用标签序列调用 drop 会从行标签（axis 0）删除值：
data.drop(['Colorado', 'Ohio'])

In [ ]:
# 通过传递 axis=1 或 axis='columns'可以删除列的值：
data.drop('two', axis=1)

In [ ]:
data.drop(['two', 'four'], axis='columns')

许多函数，如 drop，会修改 Series 或 DataFrame 的大小或形状，可以就地修 改对象，不会返回新的对象：

In [ ]:
obj = pd.Series(np.arange(5.), index=['a', 'b', 'c', 'd', 'e'])
obj.drop('c', inplace=True)
obj

小心使用 inplace，它会销毁所有被删除的数据

### 索引、选取和过滤

Series 索引（obj[...]）的工作方式类似于 NumPy 数组的索引，只不过 Series 的索引值不只是整数。下面是几个例子：

In [ ]:
obj = pd.Series(np.arange(4.), index=['a', 'b', 'c', 'd'])
obj

In [ ]:
obj['b']

In [ ]:
obj[1]

In [ ]:
obj[2:4]

In [ ]:
obj[['b', 'a', 'd']]

In [ ]:
obj[[1, 3]]

In [ ]:
obj[obj < 2]

利用标签的切片运算与普通的 Python 切片运算不同，其末端是包含的：

In [ ]:
obj['b':'c'] = 5
obj

用一个值或序列对 DataFrame 进行索引其实就是获取一个或多个列：

In [ ]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)), 
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

In [ ]:
data['two']

In [ ]:
data[['three', 'one']]

In [ ]:
# 这种索引方式有几个特殊的情况。首先通过切片或布尔型数组选取数据
data[:2]

In [ ]:
data[data['three'] > 5]

In [ ]:
#选取行的语法 data[:2]十分方便。向[ ]传递单一的元素或列表，就可选择 列。

另一种用法是通过布尔型 DataFrame（比如下面这个由标量比较运算得出的） 进行索引：

In [ ]:
data < 5

In [ ]:
data[data < 5] = 0
data

In [ ]:
# 这使得 DataFrame 的语法与 NumPy 二维数组的语法很像。

### 用 loc 和 iloc 进行选取

对于 DataFrame 的行的标签索引，我引入了特殊的标签运算符 loc 和 iloc。它 们可以让你用类似 NumPy 的标记，使用轴标签（loc）或整数索引（iloc），从 DataFrame 选择行和列的子集

In [ ]:
# 作为一个初步示例，让我们通过标签选择一行和多列：
data.loc['Colorado', ['two', 'three']]

In [ ]:
# 然后用 iloc 和整数进行选取：
data.iloc[2, [3,0,1]]

In [ ]:
data.iloc[2]

In [ ]:
data.iloc[[1, 2], [3, 0, 1]]

In [ ]:
# 这两个索引函数也适用于一个标签或多个标签的切片：
data.loc[:'Utah', 'two']

In [ ]:
data.iloc[:, :3][data.three > 5]

In [ ]:
# 所以，在 pandas 中，有多个方法可以选取和重新组合数据。对于 DataFrame， 表 5-4 进行了总结。后面会看到，还有更多的方法进行层级化索引。

笔记：在一开始设计 pandas 时，我觉得用 frame[:, col]选取列过于繁琐（也 容易出错），因为列的选择是非常常见的操作。我做了些取舍，将花式索引的 功能（标签和整数）放到了 ix 运算符中。在实践中，这会导致许多边缘情况， 数据的轴标签是整数，所以 pandas 团队决定创造 loc 和 iloc 运算符分别处理 严格基于标签和整数的索引。
ix 运算符仍然可用，但并不推荐。

###  整数索引

处理整数索引的 pandas 对象常常难住新手，因为它与 Python 内置的列表和元 组的索引语法不同。例如，你可能不认为下面的代码会出错：

In [ ]:
ser = pd.Series(np.arange(3.))
ser

In [ ]:
ser[-1]

这里，pandas 可以勉强进行整数索引，但是会导致小 bug。我们有包含 0,1,2 的索引，但是引入用户想要的东西（基于标签或位置的索引）很难：

In [ ]:
ser

另外，对于非整数索引，不会产生歧义：

In [ ]:
ser2 = pd.Series(np.arange(3.), index=['a', 'b', 'c'])
ser2[-1]

In [ ]:
# 为了进行统一，如果轴索引含有整数，数据选取总会使用标签。为了更准确， 请使用 loc（标签）或 iloc（整数）：

In [ ]:
ser[:1]

In [ ]:
ser.loc[:1]

In [ ]:
ser.iloc[:1]

### 算术运算和数据对齐

pandas 最重要的一个功能是，它可以对不同索引的对象进行算术运算。在将对 象相加时，如果存在不同的索引对，则结果的索引就是该索引对的并集。对于 有数据库经验的用户，这就像在索引标签上进行自动外连接。看一个简单的例 子：

In [ ]:
s1 = pd.Series([7.3, -2.5, 3.4, 1.5], index=['a', 'c', 'd', 'e'])
s1

In [ ]:
s2 = pd.Series([-2.1, 3.6, -1.5, 4, 3.1], index=['a', 'c', 'e', 'f', 'g'])
s2

In [ ]:
s1 + s2

自动的数据对齐操作在不重叠的索引处引入了 NA 值。缺失值会在算术运算过程 中传播。

对于 DataFrame，对齐操作会同时发生在行和列上：

In [ ]:
df1 = pd.DataFrame(np.arange(9.).reshape((3, 3)), 
                   columns=list('bcd'), 
                   index=['Ohio', 'Texas', 'Colorado'])
df1

In [ ]:
df2 = pd.DataFrame(np.arange(12.).reshape((4, 3)), 
                   columns=list('bde'), 
                   index=['Utah', 'Ohio', 'Texas', 'Oregon'])
df2

把它们相加后将会返回一个新的 DataFrame，其索引和列为原来那两个 DataFrame 的并集：

In [ ]:
df1 + df2

因为'c'和'e'列均不在两个 DataFrame 对象中，在结果中以缺省值呈现。行也 是同样。

如果 DataFrame 对象相加，没有共用的列或行标签，结果都会是空：

In [ ]:
df1 = pd.DataFrame({'A': [1, 2]})
df1

In [ ]:
df2 = pd.DataFrame({'B': [3, 4]})

In [ ]:
df2

In [ ]:
df1 - df2

### 在算术方法中填充值

在对不同索引的对象进行算术运算时，你可能希望当一个对象中某个轴标签在 另一个对象中找不到时填充一个特殊值（比如 0）：

In [ ]:
df1 = pd.DataFrame(np.arange(12.).reshape((3, 4)), 
                columns=list('abcd'))
df1

In [ ]:
df2 = pd.DataFrame(np.arange(20.).reshape((4, 5)), 
                   columns=list('abcde'))
df2.loc[1, 'b'] = np.nan
df2

In [ ]:
# 将它们相加时，没有重叠的位置就会产生 NA 值：
df1 + df2

In [ ]:
#使用 df1 的 add 方法，传入 df2 以及一个 fill_value 参数：
df1.add(df2, fill_value=0)

In [ ]:
#表 5-5 列出了 Series 和 DataFrame 的算术方法。它们每个都有一个副本，以字 母 r 开头，它会翻转参数。因此这两个语句是等价的：

In [ ]:
1 / df1

In [ ]:
df1.rdiv(1)

In [ ]:
#与此类似，在对 Series 或 DataFrame 重新索引时，也可以指定一个填充值：
df1.reindex(columns=df2.columns, fill_value=0)

### DataFrame 和 Series 之间的运算

跟不同维度的 NumPy 数组一样，DataFrame 和 Series 之间算术运算也是有明确 规定的。先来看一个具有启发性的例子，计算一个二维数组与其某行之间的 差：

In [ ]:
arr = np.arange(12.).reshape((3, 4))
arr

In [ ]:
arr[0]

In [ ]:
arr - arr[0]

当我们从 arr 减去 arr[0]，每一行都会执行这个操作。这就叫做广播 （broadcasting），附录 A 将对此进行详细讲解。DataFrame 和 Series 之间的 运算差不多也是如此：

In [ ]:
frame = pd.DataFrame(np.arange(12.).reshape((4, 3)), 
                     columns=list('bde'), 
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

In [ ]:
series = frame.iloc[0]
frame

In [ ]:
series

默认情况下，DataFrame 和 Series 之间的算术运算会将 Series 的索引匹配到 DataFrame 的列，然后沿着行一直向下广播：

In [ ]:
frame - series

如果某个索引值在 DataFrame 的列或 Series 的索引中找不到，则参与运算的两 个对象就会被重新索引以形成并集：

In [ ]:
series2 = pd.Series(range(3), index=['b','e','f'])
frame + series2

如果你希望匹配行且在列上广播，则必须使用算术运算方法。例如：

In [ ]:
series3 = frame['d']
frame

In [ ]:
series3

In [ ]:
frame.sub(series3, axis='index')

传入的轴号就是希望匹配的轴。在本例中，我们的目的是匹配 DataFrame 的行 索引（axis='index' or axis=0）并进行广播。

### 函数应用和映射

NumPy 的 ufuncs（元素级数组方法）也可用于操作 pandas 对象：

In [ ]:
frame = pd.DataFrame(np.random.randn(4, 3), 
                    columns=list('bde'),
                    index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

In [ ]:
np.abs(frame)

另一个常见的操作是，将函数应用到由各列或行所形成的一维数组上。 DataFrame 的 apply 方法即可实现此功能：

In [ ]:
f = lambda x: x.max() - x.min()
frame.apply(f)

这里的函数 f，计算了一个 Series 的最大值和最小值的差，在 frame 的每列都 执行了一次。结果是一个 Series，使用 frame 的列作为索引。

如果传递 axis='columns'到 apply，这个函数会在每行执行：

In [ ]:
frame.apply(f, axis='columns')

许多最为常见的数组统计功能都被实现成 DataFrame 的方法（如 sum 和 mean），因此无需使用 apply 方法。

传递到 apply 的函数不是必须返回一个标量，还可以返回由多个值组成的 Series：

In [ ]:
def f(x):
    return pd.Series([x.min(), x.max()], index=['min', 'max'])
frame.apply(f)

元素级的 Python 函数也是可以用的。假如你想得到 frame 中各个浮点值的格式 化字符串，使用 applymap 即可：

In [ ]:
format = lambda x: '%.2f' %x
frame.applymap(format)

之所以叫做 applymap，是因为 Series 有一个用于应用元素级函数的 map 方 法：

### 排序和排名

根据条件对数据集排序（sorting）也是一种重要的内置运算。要对行或列索引 进行排序（按字典顺序），可使用 sort_index 方法，它将返回一个已排序的新 对象：

In [ ]:
obj = pd.Series(range(4), index=['d', 'a', 'b', 'c'])
obj.sort_index()

对于 DataFrame，则可以根据任意一个轴上的索引进行排序：

In [ ]:
frame = pd.DataFrame(np.arange(8).reshape((2, 4)),
                     index=['three', 'one'], 
                     columns=['d', 'a', 'b', 'c'])
frame

In [ ]:
frame.sort_index(axis=1)

In [ ]:
frame.sort_index(axis=0)

数据默认是按升序排序的，但也可以降序排序：

In [ ]:
frame.sort_index(axis=1, ascending=False)

若要按值对 Series 进行排序，可使用其 sort_values 方法：

In [ ]:
obj = pd.Series([4, 7, -3, 2])
obj

In [ ]:
obj.sort_values()

在排序时，任何缺失值默认都会被放到 Series 的末尾：

In [ ]:
obj = pd.Series([4, np.nan, 7, np.nan, -3, 2])
obj.sort_values()

当排序一个 DataFrame 时，你可能希望根据一个或多个列中的值进行排序。将 一个或多个列的名字传递给 sort_values 的 by 选项即可达到该目的：

In [ ]:
frame = pd.DataFrame({'b': [4, 7, -3, 2], 'a': [0, 1, 0, 1]})
frame

In [ ]:
frame.sort_values(by='b')

In [ ]:
frame.sort_values(by=['a','b'])

排名会从 1 开始一直到数组中有效数据的数量。接下来介绍 Series 和 DataFrame 的 rank 方法。默认情况下，rank 是通过“为各组分配一个平均排 名”的方式破坏平级关系的：

In [ ]:
obj = pd.Series([7, -5, 7, 4, 2, 0, 4])
obj

In [ ]:
obj.rank()

也可以根据值在原数据中出现的顺序给出排名

In [ ]:
obj.rank(method='first')

In [ ]:
# 这里，条目 0 和 2 没有使用平均排名 6.5，它们被设成了 6 和 7，因为数据中标 签 0 位于标签 2 的前面。

你也可以按降序进行排名：

In [ ]:
# Assign tie values the maximum rank in the group
obj.rank(ascending=False, method='max')

DataFrame 可以在行或列 上计算排名：

In [ ]:
frame = pd.DataFrame({'b': [4.3, 7, -3, 2], 'a': [0, 1, 0, 1],
                      'c': [-2, 5, 8, -2.5]})
frame

In [ ]:
frame.rank(axis='columns')

### 带有重复标签的轴索引

直到目前为止，我所介绍的所有范例都有着唯一的轴标签（索引值）。虽然许 多 pandas 函数（如 reindex）都要求标签唯一，但这并不是强制性的。我们来 看看下面这个简单的带有重复索引值的 Series：

In [ ]:
obj = pd.Series(range(5), index=['a', 'a', 'b', 'b', 'c'])

In [ ]:
obj

索引的 is_unique 属性可以告诉你它的值是否是唯一的：

In [ ]:
obj.index.is_unique

对于带有重复值的索引，数据选取的行为将会有些不同。如果某个索引对应多 个值，则返回一个 Series；而对应单个值的，则返回一个标量值：

In [ ]:
obj['a']

In [ ]:
obj['c']

这样会使代码变复杂，因为索引的输出类型会根据标签是否有重复发生变化。

对 DataFrame 的行进行索引时也是如此：

In [ ]:
df = pd.DataFrame(np.random.randn(4, 3), 
                  index=['a', 'a', 'b', 'b'])
df

In [ ]:
df.loc['b']

## 5.3 汇总和计算描述统计

pandas 对象拥有一组常用的数学和统计方法。它们大部分都属于约简和汇总统 计，用于从 Series 中提取单个值（如 sum 或 mean）或从 DataFrame 的行或列 中提取一个 Series。跟对应的 NumPy 数组方法相比，它们都是基于没有缺失数 据的假设而构建的。看一个简单的 DataFrame：

In [58]:
df = pd.DataFrame([[1.4, np.nan], [7.1, -4.5],

            [np.nan, np.nan], [0.75, -1.3]], 
            index=['a', 'b', 'c', 'd'], 
            columns=['one', 'two'])
df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


调用 DataFrame 的 sum 方法将会返回一个含有列的和的 Series：

In [59]:
df.sum()

one    9.25
two   -5.80
dtype: float64

传入 axis='columns'或 axis=1 将会按行进行求和运算：

In [61]:
df.sum(axis=1)

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

In [62]:
# NA 值会自动被排除，除非整个切片（这里指的是行或列）都是 NA。通过 skipna 选项可以禁用该功能：
df.mean(axis='columns', skipna=False)

a      NaN
b    1.300
c      NaN
d   -0.275
dtype: float64

In [63]:
df.mean()

one    3.083333
two   -2.900000
dtype: float64

有些方法（如 idxmin 和 idxmax）返回的是间接统计（比如达到最小值或最大 值的索引）：

In [64]:
df.idxmax()

one    b
two    d
dtype: object

In [65]:
df.idxmin()

one    d
two    b
dtype: object

另一些方法则是累计型的：

In [66]:
df.cumsum()

,one,two
a,1.40,NaN
b,8.50,-4.5
c,NaN,NaN
d,9.25,-5.8


还有一种方法，它既不是约简型也不是累计型。describe 就是一个例子，它用 于一次性产生多个汇总统计：

In [67]:
df.describe()

,one,two
count,3.000000,2.000000
mean,3.083333,-2.900000
std,3.493685,2.262742
min,0.750000,-4.500000
25%,1.075000,-3.700000
50%,1.400000,-2.900000
75%,4.250000,-2.100000
max,7.100000,-1.300000


对于非数值型数据，describe 会产生另外一种汇总统计：

In [68]:
obj = pd.Series(['a', 'a','b','c'] * 4)
obj

0     a
1     a
2     b
3     c
4     a
5     a
6     b
7     c
8     a
9     a
10    b
11    c
12    a
13    a
14    b
15    c
dtype: object

In [69]:
obj.describe()

count     16
unique     3
top        a
freq       8
dtype: object

### 相关系数与协方差

有些汇总统计（如相关系数和协方差）是通过参数对计算出来的。我们来看几 个 DataFrame，它们的数据来自 Yahoo!Finance 的股票价格和成交量，使用的 是 pandas-datareader 包

In [ ]:
# 我使用 pandas_datareader 模块下载了一些股票数据：
import pandas_datareader as web
all_data = {ticker: web.get_data_yahoo(ticker)
                   for ticker in ['AAPL', 'IBM', 'MSFT', 'GOOG']}
price = pd.DataFrame({ticker: data['Adj Close']
                     for ticker, data in all_data.items()})
volume = pd.DataFrame({ticker: data['Volume']
                      for ticker, data in all_data.items()})
#注意：此时 Yahoo! Finance 已经不存在了，因为 2017 年 Yahoo!被 Verizon 收 购了。参阅 pandas-datareader 文档，可以学习最新的功能。

现在计算价格的百分数变化，时间序列的操作会在第 11 章介绍：

In [ ]:
returns = price.pct_change()
#returns
returns.tail()